In [203]:
# Import library
import pandas as pd
import numpy as np
import json
import datetime
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

In [21]:
# Load json data from file
data = []
with open('dataset/sgcircuitbreaker.json', encoding="utf8") as f:
    for line in f:
        data.append(json.loads(line))

In [259]:
# Import json data into dataframe
df = pd.DataFrame(data)
df.head()

,_id,created_at,id,screen_name:,lang,is_quote_status,is_retweet_status,full_text,quote_text
0,{'$oid': '5eae613c65da26ed42446310'},{'$date': {'$numberLong': '1588486453000'}},{'$numberLong': '1256829397778132992'},Ashy_roz,en,True,False,Morning Peepos @MdAnde1 @SyiqinKmz,Good morning...\n#Singapore \n#circuitbreaker ...
1,{'$oid': '5eae614b65da26ed42446311'},{'$date': {'$numberLong': '1588486468000'}},{'$numberLong': '1256829460193689601'},harshdass11,en,False,False,"There is proof in📗 quraan sharif, who has comp...",
2,{'$oid': '5eae616665da26ed42446312'},{'$date': {'$numberLong': '1588486496000'}},{'$numberLong': '1256829574723383296'},harshdass11,en,False,True,Is creator🤴 of all the universe🌍\nAlmighty God...,
3,{'$oid': '5eae616a65da26ed42446313'},{'$date': {'$numberLong': '1588486500000'}},{'$numberLong': '1256829592066801665'},staciechan,en,False,False,Day 26 #Covid19 #circuitbreakersg: unleashing ...,
4,{'$oid': '5eae618465da26ed42446314'},{'$date': {'$numberLong': '1588486525000'}},{'$numberLong': '1256829698048274434'},SStretchsg,en,False,False,#𝘿𝙚𝙡𝙞𝙫𝙚𝙧𝙮𝙍𝙞𝙙𝙚𝙧𝙨 are your shoulder stiff from a...,


### Things to clean:
- Checking any missing/ duplicated value
- Extract data from json object inside cell for columns `_id`, `created_at`, `id`
- Rename columns

### Text cleaning:
- For `full_text`, `quote_text` column, text preprocessing is required for sentiment analysis
- Convert to lowercase, remove noise and stopword, tokenization

Before data cleaning, create another copy of dataframe.

In [260]:
df_clean = df.copy()

#### Checking missing value

In [261]:
# Checking missing value
df_clean.isnull().sum()

_id                  0
created_at           0
id                   0
screen_name:         0
lang                 0
is_quote_status      0
is_retweet_status    0
full_text            0
quote_text           0
dtype: int64

No missing value is found.

#### Extract json data

May Refer these links to see how to extract json data
- [w3school](https://www.w3schools.com/python/python_json.asp)
- [Stack Overflow](https://stackoverflow.com/questions/42354001/python-json-object-must-be-str-bytes-or-bytearray-not-dict/42354033)

In [262]:
# Extract json object from _id column
df_clean['_id'] = df_clean['_id'].apply(lambda x: json.loads(json.dumps(x))['$oid'])

In [263]:
# Extract json object from id column
df_clean['id'] = df_clean['id'].apply(lambda x: json.loads(json.dumps(x))['$numberLong'])

In [264]:
# Extract json object from id column
df_clean['created_at'] = df_clean['created_at'].apply(lambda x: json.loads(json.dumps(x))['$date']['$numberLong'])

In [168]:
# Check value
df_clean.sample(10)

,_id,created_at,id,screen_name:,lang,is_quote_status,is_retweet_status,full_text,quote_text
1991,5eaf7a92b194fb305583216d,1588558476000,1257131481358454785,mimicrassy,en,False,True,What were these 7 people thinking?? #Stayhomef...,
4460,5eb0b53cb194fb3055832b12,1588639030000,1257469350069645312,Samira3787,ar,False,False,@AkasdMabdah فوغا مسويه عروض جدا رهيبه واستخدم...,
5202,5eb14ea0b194fb3055832df8,1588678298000,1257634051902418945,ChoLiu92,und,False,True,200505 #CB\n🐶🐶 https://t.co/ZK8j8fcL2U,
3743,5eb03e73b194fb3055832845,1588608620000,1257341802769797121,dassi_aarti,en,False,True,It is very rare to find 🔍an tatv darshi saint....,
666,5eae957f65da26ed424465aa,1588499833000,1256885515439104000,jessie_36milk,ja,False,True,⚽️Ｊリーグ⚽️\n\n【横浜】再開後の過密日程で存在感を発揮できるか？ ニューカマー山本義...,
6498,5eb194a7b194fb3055833308,1588696224000,1257709240241831936,Manohar37840440,en,False,True,#Say_No_To_Alcohol\nAccording to the constitut...,
1890,5eaf68eeb194fb3055832108,1588553960000,1257112541580324864,cafe_rosemary,ko,False,True,"“논문을 미리 읽은 같은 학교에 다니는 지인이 말없이 최씨를 끌어안고 “언니, 여태...",
869,5eaeb617b194fb3055831d0b,1588508176000,1256920510392201220,syfeka,in,True,True,"Sometimes, you genuinely forgot. Mak aku gi pa...",How come the mall security outside the bakery ...
3444,5eb03800b194fb305583271a,1588606969000,1257334877764149251,bhawnamadam,en,False,True,Brahma🤴 Vishnu Mahesh is not immortal. They al...,
8860,5eb40177d16fd09c375d5e5f,1588855153000,1258375836836618240,kevin_0j0,zh,False,True,#凱文的鎖屌計畫 #Day7\n\n統計結果出來啦～\n第一次鎖屌就要被鎖 #123天🔒😰\...,


#### Checking duplicated value

In [265]:
# Checking duplicated value
df_clean.duplicated().sum()

0

No duplicate row is found.

#### Convert Unix timestamp to datetime

In [266]:
# Remove last 3 characters '000' from unix string (Should be 10 digits only)
# Convert created_at values from object to string first before it can convert to timestamp format
df_clean['created_at'] = df_clean['created_at'].apply(lambda x: x[:-3])
df_clean['created_at'] = df_clean['created_at'].astype(np.int64)

In [267]:
# Convert unix timestamp to datetime format
df_clean['created_at'] = df_clean['created_at'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%dT%H:%M:%SZ'))

In [268]:
df_clean['created_at'] = pd.to_datetime(df_clean['created_at'])

#### Drop Columns
`_id` is id from mongo database, thus, it is not required in this project and could be dropped.

In [269]:
df_clean.drop('_id', axis =1, inplace =True)

#### Rename Columns
- Change `id` to `user_id`  
- Change `screen_name:` to `username`
- Change `lang` to `language`

In [270]:
df_clean.rename(columns = {'id':'user_id', 'screen_name:':'username', 'lang':'language'}, inplace = True)

#### Define Data Cleaning Functions
For tweet cleaning, may refer [here](https://towardsdatascience.com/extracting-twitter-data-pre-processing-and-sentiment-analysis-using-python-3-0-7192bd8b47cf).

In [271]:
# Convert all characters to lowercase
def to_lowercase(text):
    text  = text.lower()
    return text

In [272]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [273]:
# Remove Emoji
def remove_emoji(text):
    text = emoji_pattern.sub(r'', text)
    return text

In [295]:
# Remove Mentions
def remove_mention(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    return text

In [341]:
# Remove Hashtag
def remove_hashtag(text):
    text = re.sub(r'#\w+', '', text)
    return text

In [333]:
# Remove URL
def remove_url(text):
    text = re.sub(r'\b(?:(?:https?|ftp)://)?\w[\w-]*(?:\.[\w-]+)+\S*', ' ', text)
    return text

In [275]:
# Replace consecutive non-ASCII characters with a space
def replace_nonASCII(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    return text

In [277]:
# Remove punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [278]:
# Remove stopwords
stopwordsList = set(stopwords.words('english')) 
def remove_stopwords(text):
    text = [word for word in text if word not in stopwordsList]
    return text

In [279]:
# Tokenize words
def tokenize(text):
    word_tokens = word_tokenize(text)
    return word_tokens

In [344]:
# Combine all the functions
def datapreprocessing(review):
    
    # Convert the text into lowercase
    review = to_lowercase(review)
    
    # Clean tweet
    review = remove_mention(review)
    review = remove_url(review)
    review = remove_hashtag(review)
    review = remove_emoji(review)
    review = replace_nonASCII(review)
        
    # Remove punctuation
    review = remove_punct(review)
    
    # Tokenization
    review = word_tokenize(review)
    
    # Remove stopwords
    review = remove_stopwords(review)
    
    #return review
    return " ".join(review)

Before text preprocessing, we would like to remain all the text in **English** only.

In [345]:
df_clean = df_clean.loc[df_clean['language'] == 'en']

In [346]:
cols = ['full_text', 'quote_text']

for c in cols:
    df_clean[c] = df_clean[c].apply(lambda x: datapreprocessing(x))

In [348]:
df_clean.sample(10)

,created_at,user_id,username,language,is_quote_status,is_retweet_status,full_text,quote_text
7190,2020-05-05 18:18:47+00:00,1257736514102067200,Sahil39399351,en,False,True,children whose fathersare intoxicated natural ...,
5096,2020-05-05 10:30:41+00:00,1257618713198485504,jas0nsg,en,True,False,infections,new covid cases singapore taking total past
89,2020-05-03 06:23:44+00:00,1256831789433122816,way_of_worship,en,False,True,brahma vishnu mahesh complete god know complet...,
8938,2020-05-07 14:35:40+00:00,1258405142669357056,mrbrown,en,False,False,st day kim huat searching elusive items since ...,
5079,2020-05-05 10:11:16+00:00,1257613828814180352,CoachColombi,en,False,True,ar back lab wish college summer camps check db...,
7710,2020-05-06 08:02:31+00:00,1257943812489478144,SajjanKaur1,en,False,True,children whose fathersare intoxicated natural ...,
3062,2020-05-04 14:24:00+00:00,1257315042325704706,keyboardcowgirl,en,False,False,law minister made fb post come right clarify w...,
8321,2020-05-06 23:25:33+00:00,1258176102767276035,bw_wannabe,en,False,True,ok heck bought cream cheese fairprice,
8449,2020-05-07 03:34:45+00:00,1258238816181248001,BbmsBroncos,en,True,False,broncos fun actions take spread fun,stayathome bingo rounds search boards pick squ...
9155,2020-05-08 01:42:29+00:00,1258572952599060480,sarahcottle,en,False,False,coworker may need haircut complaints dulwich,


In [349]:
df_clean.to_csv('dataset/data_clean.csv', index = False)